In [71]:
%pip uninstall -qqqy google-generativeai google-ai-generativelanguage
%pip install -qU "langchain[google-genai]" langchain langgraph>0.2.27 langchain_community pypdf faiss-cpu langchain-tavily langchain-huggingface langgraph-checkpoint-sqlite tiktoken langchain-neo4j neo4j langchain-experimental
import os
import getpass
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model
load_dotenv()
# .env should also have TAVILY_API_KEY
os.environ["USER_AGENT"] = "MyLangChainBot/1.0 (+https://example.com/contact)"
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "default"
if not os.environ.get("TAVILY_API_KEY"):
  os.environ["TAVILY_API_KEY"] = getpass.getpass("Enter API key for Tavily: ")
if not os.environ.get("GEMINI_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")
else:
  os.environ["GOOGLE_API_KEY"] = os.environ.get("GEMINI_API_KEY")
if not os.environ.get("HF_TOKEN"):
  os.environ["HF_TOKEN"] = getpass.getpass("Enter API key for Hugging Face: ")
llm = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

In [72]:
from langchain.prompts import PromptTemplate

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)
p = prompt_template_name.format(cuisine="Italian")
print(p)

# print(llm.predict(p))

I want to open a restaurant for Italian food. Suggest a fency name for this.


In [73]:
# Simple Chain

from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt_template_name = PromptTemplate(
    input_variables=['cuisine'],
    template="Suggest a fancy name for a restaurant that serves {cuisine} food."
)

prompt_template_items = PromptTemplate(
    input_variables=['restaurant_name'],
    template="Suggest some menu items for {restaurant_name}."
)

full_chain = (
    prompt_template_name
    | llm
    | StrOutputParser()
    | prompt_template_items
    | llm
    | StrOutputParser()
)

content = full_chain.invoke({"cuisine": "Indian"})
print(content)

Here are some menu item suggestions for a fancy Indian restaurant, keeping in mind the elegant, royal, and modern names you've provided. The descriptions aim for sophistication and highlight unique ingredients or preparation methods.

---

## **The Gilded Lotus / Maharaja's Table / The Saffron & Silk**
*An Exquisite Culinary Journey Through India*

---

### **Apertifs & Small Plates (Starters)**

*   **Tandoori Scallops with Saffron-Lime Butter (Seafood)**
    *   Pan-seared jumbo scallops, delicately marinated in a hint of tandoori spice, served on a bed of spiced pea puree with a vibrant saffron-lime butter emulsion and microgreens.
*   **Deconstructed Samosa Chaat (Vegetarian)**
    *   Crispy pastry shards artfully arranged with spiced potato and pea crumble, spheres of sweet tamarind gel, mint foam, and a drizzle of chilled yogurt, garnished with pomegranate arils.
*   **Lamb Seekh Kebab "Cigars" with Pistachio Dust & Rose Petal Raita (Meat)**
    *   Finely minced lamb kebabs, su

In [74]:
# Sequential Chain
from pprint import pprint
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel

prompt_template_name = PromptTemplate(
    input_variables=['cuisine'],
    template="I want to open a restaurant for {cuisine} food. Suggest a fancy name for this."
)

prompt_template_items = PromptTemplate(
    input_variables=['restaurant_name'],
    template="Suggest some menu items for {restaurant_name}."
)

name_chain = (
    prompt_template_name
    | llm
    | StrOutputParser()
)

full_chain = (
    RunnablePassthrough.assign(restaurant_name=name_chain)
    | {
        "restaurant_name": lambda x: x['restaurant_name'],
        "menu_items": prompt_template_items | llm | StrOutputParser()
    }
)

content = full_chain.invoke({"cuisine": "Indian"})

pprint(content)

{'menu_items': 'Here are some menu item suggestions for your fancy Indian '
               'restaurant, drawing inspiration from the elegant and regal '
               "names you've chosen. The goal is to elevate classic Indian "
               'dishes with premium ingredients, refined techniques, and '
               'sophisticated presentation.\n'
               '\n'
               '---\n'
               '\n'
               "### **A Culinary Journey Through India's Heritage**\n"
               '\n'
               '#### **I. Regal Beginnings (Appetizers & Small Plates)**\n'
               '\n'
               '1.  **The Kohinoor Scallops**\n'
               '    *   *Description:* Pan-seared jumbo sea scallops, '
               'delicately spiced with saffron and star anise, served atop a '
               'creamy coconut-lime moilee sauce with microgreens.\n'
               "2.  **Maharaja's Galouti Croquettes**\n"
               '    *   *Description:* Melt-in-your-mouth lamb galouti,

In [79]:
# Tavily Search and PythonREPLTool
from langchain_core.tools import tool
from langchain_tavily import TavilySearch
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

@tool
def tavily_search_tool(query: str) -> str:
    """A search engine that provides up-to-date information by searching the web.
    Useful for questions about current events, recent data, or facts that may have changed."""
    return TavilySearch().invoke(query)

@tool
def python_repl_tool(code: str) -> str:
    """A Python shell. Use this to execute valid Python commands.
    Useful for performing calculations and data manipulation."""
    return PythonREPLTool().run(code)

tools = [tavily_search_tool, python_repl_tool]

prompt = ChatPromptTemplate.from_messages([
    ("system",
     "You are an expert assistant with access to powerful tools. You must use the 'tavily_search' tool whenever the user asks for current information, facts, or data, especially if the query refers to a specific year or recent events. For any mathematical calculations, use the 'python_repl' tool. Do not use your internal knowledge for questions that require real-time data."
    ),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

agent = create_tool_calling_agent(llm, tools, prompt)

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True
)

result = agent_executor.invoke({"input": "What was the GDP PPP of China in 2025? add 5 trillions to it and answer in trillion"})

print(result['output'])




> Entering new AgentExecutor chain...

Invoking: `tavily_search_tool` with `{'query': 'GDP PPP of China in 2025'}`


{'query': 'GDP PPP of China in 2025', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://www.worldeconomics.com/GrossDomesticProduct/Real-GDP-PPP/Saudi%20Arabia.aspx/China.aspx', 'title': 'China GDP PPP 2024 Estimate and 2025 Projection', 'content': "China's GDP PPP is projected to be $34.3 Trillion by the end of 2025, from an estimated $32.8 Trillion in 2024. Explore and download the latest economic", 'score': 0.9646194, 'raw_content': None}, {'url': 'https://www.worldeconomics.com/Processors/Economics-Countries-GrossDomesticProduct.aspx?Country=China', 'title': 'China GDP: $43.204 trillion', 'content': "World Economics estimates China's 2025 GDP at $43.204 trillion in PPP terms (Purchasing Power Parity) and an initial estimate of $45.148 trillion for 2026.", 'score': 0.9539705, 'raw_content': None}, {'url': 'https://tradingeconomi

In [82]:
# Wikipedia and PythonREPLTool
from langchain_core.tools import tool
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools.wikipedia.tool import WikipediaQueryRun
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0)

@tool
def wikipedia_search_tool(query: str) -> str:
    """A search engine that provides up-to-date information by searching the web.
    Useful for questions about current events, recent data, or facts that may have changed."""
    wrapper = WikipediaAPIWrapper(top_k_results=1)
    return WikipediaQueryRun(api_wrapper=wrapper).invoke(query)

@tool
def python_repl_tool(code: str) -> str:
    """A Python shell. Use this to execute valid Python commands.
    Useful for performing calculations and data manipulation."""
    return PythonREPLTool().run(code)

tools = [wikipedia_search_tool, python_repl_tool]

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant with access to Wikipedia and Python. Use Wikipedia for factual lookups and Python for calculations."),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

agent = create_tool_calling_agent(llm, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

result = agent_executor.invoke({
    "input": "When was Bill Gates born? What is his age right now in 2025?"
})

print(result["output"])




> Entering new AgentExecutor chain...

Invoking: `wikipedia_search_tool` with `{'query': 'Bill Gates'}`


Page: Bill Gates
Summary: William Henry Gates III    (born October 28, 1955) is an American businessman and philanthropist. A pioneer of the microcomputer revolution of the 1970s and 1980s, he co-founded the software company Microsoft in 1975 with his childhood friend Paul Allen. Following the company's 1986 initial public offering (IPO), Gates became a billionaire in 1987—then the youngest ever, at age 31. Forbes magazine ranked him as the world's wealthiest person for 18 out of 24 years between 1995 and 2017, including 13 years consecutively from 1995 to 2007. He became the first centibillionaire in 1999, when his net worth briefly surpassed $100 billion. According to Forbes, as of May 2025, his net worth stood at US$115.1 billion, making him the thirteenth-richest individual in the world.
Born and raised in Seattle, Washington, Gates was privately educated at Lakeside School, 

In [80]:
from typing import List
from pydantic import Field
from langchain_core.messages import BaseMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder


class LimitedChatMessageHistory(ChatMessageHistory):
    k: int = Field(default=1)

    def add_messages(self, messages: List[BaseMessage]) -> None:
        super().add_messages(messages)
        # Each turn = 2 messages (Human + AI)
        max_messages = self.k * 2
        self.messages = self.messages[-max_messages:]

prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Suggest a fancy name for a restaurant that serves {cuisine} food."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{cuisine}")
])

chain = prompt_template | llm | StrOutputParser()

store = {}

def get_limited_session_history(session_id: str) -> LimitedChatMessageHistory:
    if session_id not in store:
        store[session_id] = LimitedChatMessageHistory(k=2)
    return store[session_id]

with_memory_chain = RunnableWithMessageHistory(
    chain,
    get_limited_session_history,
    input_messages_key="cuisine",
    history_messages_key="chat_history"
)

config = {"configurable": {"session_id": "restaurant_session"}}
name1 = with_memory_chain.invoke(
    {"cuisine": "Mexican"},
    config=config
)
print("Mexican restaurant name:", name1)

name2 = with_memory_chain.invoke(
    {"cuisine": "Italian"},
    config=config
)
print("\n\n\nItalian restaurant name:", name2)

name3 = with_memory_chain.invoke(
    {"cuisine": "Japanese"},
    config=config
)
print("\n\n\nJapanese restaurant name:", name3)

print("\nFinal session history (should have only 2 messages):")
for message in store["restaurant_session"].messages:
    print(f"[{message.type}] {message.content}")


Mexican restaurant name: Okay, let's get fancy with some Mexican restaurant names! I'll aim for elegant, sophisticated, and evocative.

Here are some suggestions, broken down by style:

---

**I. Elegant Spanish & Poetic:**
*   **Alma y Fuego:** (Soul and Fire) - Suggests passion, tradition, and cooking with heart.
*   **Noche Azul:** (Blue Night) - Evokes a sophisticated, romantic evening.
*   **El Corazón de México:** (The Heart of Mexico) - Classic, warm, and authentic.
*   **Sabor de la Noche:** (Flavor of the Night) - Mysterious and enticing.
*   **Las Raíces:** (The Roots) - Implies deep tradition and authentic origins.
*   **La Mesa de Oro:** (The Golden Table) - Suggests a luxurious dining experience.
*   **Cielo y Tierra:** (Sky and Earth) - Evokes natural ingredients and a grounded, yet elevated, experience.
*   **Puebla Escondida:** (Hidden Puebla) - Suggests a secret gem, perhaps specializing in mole.

---

**II. Modern & Chic:**
*   **Mesa Fina:** (Fine Table) - Direct, el